# Bluesky Hexapod Operation

Cleaning up bluesky routines and testing sample stage scan before commissioning

Time:  October 2025

Contact: awojdyla@lbl.gov, francisho@lbl.gov, kmchan@lbl.gov, seijdeleopn@lbl.gov

## Set Up

### Import libraries

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg', force=True)    # Use Qt5Agg backend for interactive plots
import matplotlib.pyplot as plt
plt.ion()   # Enable interactive mode

import h5py
import hdf5plugin   # For compression support

from PIL import Image   # For Tif image display

from IPython.display import Image, display     # For displaying images inline

import time
from time import sleep as time_sleep
import io
import os
from datetime import datetime

In [ ]:
import epics
import bluesky
from bluesky import RunEngine

from bluesky.callbacks import LiveTable, LivePlot
from bluesky.callbacks.best_effort import BestEffortCallback

from bluesky.plans import scan, count, list_scan, grid_scan 

from databroker import Broker

RE = RunEngine({})

# Insert all metadata/data captured into db.
db = Broker.named('temp')
RE.subscribe(db.insert)

# Setup BestEffortCallback for automatic plots and tables
bec = BestEffortCallback()
RE.subscribe(bec)   # Default live feedback

1

## Connecting to Hexapod

### Connect and configure Pilatus detector

In [ ]:
# =============================================================================
# PILATUS DETECTOR CONFIGURATION
# =============================================================================

from ophyd import ADComponent, ImagePlugin, PilatusDetector, SingleTrigger, EpicsSignalRO
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite
from ophyd.areadetector.plugins import HDF5Plugin_V34

# CONFIGURATION PARAMETERS
DETECTOR_PREFIX = "13PIL1:"
PILATUS_FILES_ROOT = "/mnt/data531"
BLUESKY_FILES_ROOT = "/mnt/data531"
TEST_IMAGE_DIR = "test/pilatus/%Y/%m/%d/"

# Detector settings
EXPOSURE_TIME = 0.1      # seconds
ACQUIRE_PERIOD = 0.105   # seconds

# DETECTOR CLASSES
class MyHDF5Plugin(FileStoreHDF5IterativeWrite, HDF5Plugin_V34): 
    """Custom HDF5 plugin with file store integration"""
    pass

class MyPilatusDetector(SingleTrigger, PilatusDetector):
    """Pilatus detector with HDF5 file writing capability"""
    image = ADComponent(ImagePlugin, "image1:")
    hdf1 = ADComponent(
        MyHDF5Plugin,
        "HDF1:",
        write_path_template=os.path.join(PILATUS_FILES_ROOT, TEST_IMAGE_DIR),
        read_path_template=os.path.join(BLUESKY_FILES_ROOT, TEST_IMAGE_DIR),
    )

# DETECTOR INITIALIZATION

print("Initializing Pilatus detector...")

# Create detector instance
det = MyPilatusDetector(DETECTOR_PREFIX, name="det")

# Configure detector settings
det.hdf1.create_directory.put(-5)
det.cam.stage_sigs["image_mode"] = "Single"
det.cam.stage_sigs["num_images"] = 1
det.cam.stage_sigs["acquire_time"] = EXPOSURE_TIME
det.cam.stage_sigs["acquire_period"] = ACQUIRE_PERIOD
det.hdf1.stage_sigs["lazy_open"] = 1
det.hdf1.stage_sigs["compression"] = "LZ4"
det.hdf1.stage_sigs["file_template"] = "%s%s_%3.3d.h5"
del det.hdf1.stage_sigs["capture"]
det.hdf1.stage_sigs["capture"] = 1

print("\nPilatus detector configured successfully!")
print(f"    PV prefix: {DETECTOR_PREFIX}")
print(f"    Exposure time: {EXPOSURE_TIME}s")
print(f"    Acquire period: {ACQUIRE_PERIOD}s")

# Function to convert ASCII array to string
def ascii_array_to_string(ascii_array):
    try:
        if isinstance(ascii_array, (list, np.ndarray)):
            arr = np.array(ascii_array)
            chars = [chr(int(x)) for x in arr if x != 0 and x < 128]
            return ''.join(chars)
        else:
            return str(ascii_array)
    except:
        return str(ascii_array)
    
# Test connectivity and show next file
try:
    file_path_signal = EpicsSignalRO(f'{DETECTOR_PREFIX}cam1:FilePath_RBV')
    file_name_signal = EpicsSignalRO(f'{DETECTOR_PREFIX}cam1:FileName_RBV')
    file_number_signal = EpicsSignalRO(f'{DETECTOR_PREFIX}cam1:FileNumber_RBV')
    time.sleep(1)
    
    current_path = ascii_array_to_string(file_path_signal.get())
    current_name = ascii_array_to_string(file_name_signal.get())
    next_file_number = file_number_signal.get()
    
    # Construct next filename
    next_filename = f"{current_path}{current_name}{next_file_number:03d}.tif"

    print("\nDetector connectivity confirmed!")
    print(f"    File path: {current_path}")
    print(f"    Base name: {current_name}")
    print(f"    Next file number: {next_file_number}")
    print(f"    Next file will be: {next_filename}")
    
except Exception as e:
    print(f"⚠️  Connectivity test failed: {e}")

Initializing Pilatus detector...

Pilatus detector configured successfully!
    PV prefix: 13PIL1:
    Exposure time: 0.1s
    Acquire period: 0.105s

Detector connectivity confirmed!
    File path: /mnt/data531/
    Base name: 20250702_alien
    Next file number: 0
    Next file will be: /mnt/data531/20250702_alien000.tif


In [ ]:
# =============================================================================
# Defines LiveTable and file tracking callbacks.
# =============================================================================

from bluesky.callbacks import CallbackBase

class LiveTableWithPoint(LiveTable):
    """Enhanced LiveTable with point numbers and clean columns"""
    
    def event(self, doc):
        # Add point number as a data column
        doc = dict(doc)
        if 'data' not in doc:
            doc['data'] = {}
        doc['data'] = dict(doc['data'])
        doc['data']['point'] = doc['seq_num']
        
        # Remove setpoint columns for cleaner display
        keys_to_remove = [key for key in doc['data'].keys() if 'user_setpoint' in key]
        for key in keys_to_remove:
            del doc['data'][key]
        
        super().event(doc)

class FilePathCallback(CallbackBase):
    """Tracks file paths and provides scan summary"""
    
    def __init__(self):
        self.scan_data = []
        
    def start(self, doc):
        print("\n📸 Image Files:")
        print("-" * 80)
        self.scan_data = []
        
    def event(self, doc):
        try:
            point_num = doc['seq_num']
            x_pos = doc['data'].get('sample stage horizontal', 0)
            y_pos = doc['data'].get('sample stage vertical', 0)
            
            # Get file information
            file_number_signal = EpicsSignalRO(f'{DETECTOR_PREFIX}cam1:FileNumber_RBV')
            file_path_signal = EpicsSignalRO(f'{DETECTOR_PREFIX}cam1:FilePath_RBV')
            file_name_signal = EpicsSignalRO(f'{DETECTOR_PREFIX}cam1:FileName_RBV')
            
            current_file_num = file_number_signal.get() - 1
            file_path = ascii_array_to_string(file_path_signal.get())
            file_name = ascii_array_to_string(file_name_signal.get())
            full_path = f"{file_path}{file_name}{current_file_num:03d}.tif"
            
            print(f"Point {point_num}: {full_path}")
            
            self.scan_data.append({
                'point': point_num,
                'x_pos': x_pos,
                'y_pos': y_pos,
                'full_path': full_path
            })
            
        except Exception as e:
            print(f"Point {doc['seq_num']}: Error - {e}")
            self.scan_data.append({
                'point': doc['seq_num'],
                'x_pos': doc['data'].get('sample stage horizontal', 0),
                'y_pos': doc['data'].get('sample stage vertical', 0),
                'full_path': f"Error: {e}"
            })
    
    def stop(self, doc):
        print("-" * 80)
        print("✅ Scan completed!\n")
        self.print_scan_summary()
    
    def print_scan_summary(self):
        print("\n" + "="*100)
        print("📋 SCAN SUMMARY")
        print("="*100)
        print(f"{'Point':<8} {'X Position':<12} {'Y Position':<12} {'Full File Path':<60}")
        print("-" * 100)
        
        for data in self.scan_data:
            print(f"{data['point']:<8} "
                  f"{data['x_pos']:<12.3f} "
                  f"{data['y_pos']:<12.3f} "
                  f"{data['full_path']:<60}")
        
        print("-" * 100)
        print(f"Total points: {len(self.scan_data)}")
        print("="*100)

print("✅ Scan classes ready!")

✅ Scan classes ready!


### Connecting to EpicsMotor

We had some issues here witht the picoscale interformeter -- we need to use caproto to talk to it (ophyd's motor does not work directly out of the box)

from: https://nsls-ii.github.io/bluesky/tutorial.html

```
from ophyd import EpicsMotor
nano_top_x = EpicsMotor('XF:31ID-ES{Dif:Nano-Ax:TopX}Mtr', name='nano_top_x')
```
see also:

https://blueskyproject.io/ophyd/generated/ophyd.epics_motor.EpicsMotor.html#:~:text=class%20ophyd.epics_motor.EpicsMotor%28prefix%3D%27%27%2C%20%2A%2C%20name%2C%20kind%3DNone%2C%20read_attrs%3DNone%2C%20configuration_attrs%3DNone%2C%20parent%3DNone%2C,are%20passed%20through%20to%20the%20base%20class%2C%20Positioner




In [4]:
from ophyd import EpicsMotor
import ophyd
ophyd.set_cl('caproto')

sample_x_mm = ophyd.EpicsMotor('bl531_xps2:sample_x_mm', name='sample stage horizontal')
sample_y_mm = ophyd.EpicsMotor('bl531_xps2:sample_y_mm', name='sample stage vertical')
beamstop_diode = ophyd.EpicsSignal('bl201-beamstop:current', name='beamstop diode')

#this works
sample_x_mm.wait_for_connection(timeout=2.0)
sample_x_mm.read()
sample_y_mm.wait_for_connection(timeout=2.0)
sample_y_mm.read()
beamstop_diode.wait_for_connection(timeout=2.0)
beamstop_diode.read()

{'beamstop diode': {'value': 0.0, 'timestamp': 631152000.0}}

In [5]:
from bluesky.plans import scan
# just a motor scan, no detectors
RE(scan([], sample_x_mm, 1, 2, sample_y_mm, 1,2, 11))

#works, but does a weird scan
#https://nsls-ii.github.io/bluesky/generated/bluesky.plans.grid_scan.html#bluesky.plans.grid_scan


('f4442c1a-1e11-4583-8f5f-dbfb98ea40e3',)

In [7]:
from bluesky.plans import grid_scan

RE(grid_scan([], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('f1183da2-0559-4b45-9fac-280258be189d',)

In [6]:
from bluesky.plans import grid_scan

RE(grid_scan([beamstop_diode], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('d4536f2f-24d6-4115-83d0-a1f2d4851b01',)

In [7]:
header = db[-1]
header.table()

,time,beamstop diode,sample stage horizontal,sample stage horizontal_user_setpoint,sample stage vertical,sample stage vertical_user_setpoint
seq_num,,,,,,
1,2025-02-25 18:34:26.733349085,0.0,1.0,1.0,1.0,1.0
2,2025-02-25 18:34:28.219246864,0.0,1.0,1.0,1.5,1.5
3,2025-02-25 18:34:29.700576067,0.0,1.0,1.0,2.0,2.0
4,2025-02-25 18:34:31.878351688,0.0,1.5,1.5,1.0,1.0
5,2025-02-25 18:34:33.361911058,0.0,1.5,1.5,1.5,1.5
6,2025-02-25 18:34:34.838891983,0.0,1.5,1.5,2.0,2.0
7,2025-02-25 18:34:37.007189512,0.0,2.0,2.0,1.0,1.0
8,2025-02-25 18:34:38.487745762,0.0,2.0,2.0,1.5,1.5
9,2025-02-25 18:34:39.976714849,0.0,2.0,2.0,2.0,2.0


In [17]:
# some issues here -- not critical
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
# Send all metadata/data captured to the BestEffortCallback.
bec_token = RE.subscribe(bec)
# Make plots update live while scans run.
from bluesky.utils import install_kicker


/usr/local/lib/python3.10/dist-packages/bluesky/utils/__init__.py:934: UserWarning: bluesky.utils.install_qt_kicker is no longer necessary and has no effect. Please remove your use of it. It may be removed in a future release of bluesky.
  warnings.warn("bluesky.utils.install_qt_kicker is no longer necessary and "


In [21]:
RE.unsubscribe(bec_token)
RE(grid_scan([], sample_x_mm, 1, 2, 3, sample_y_mm, 1,2,3))

('f59ba064-15f1-4e77-a008-19e88331a6fc',)

In [ ]:
RE(scan(dets, motors, 25, 28, 61))

In [8]:
epics.caput('DMC02:E.CNEN', 0)
epics.caput('DMC02:F.CNEN', 0)
epics.caput('DMC02:G.CNEN', 0)
epics.caput('DMC02:H.CNEN', 0)

1